<a href="https://colab.research.google.com/github/AfricArxiv/repo-dev/blob/master/osf_preprints_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import json
import time

ti = time.time()

preprintsApiUrl = "https://api.osf.io/v2/providers/preprints/africarxiv/preprints/?page[size]=57"
preprintsRequest = requests.get( url = preprintsApiUrl )
preprintsRequestContent = json.loads( preprintsRequest.content )

for preprint in preprintsRequestContent['data']:
  print( preprint['attributes']['date_published'] )
  print( preprint['attributes']['title'] )
  #print( preprint['attributes']['description'] )
  print( preprint['links']['html'] )
  print( preprint['links']['preprint_doi'] )
  
  for subject in preprint['attributes']['subjects']:
    for subj in subject:
      print( subj['text'] )
    print( '\n' )
    
  authorApiUrl = preprint['relationships']['contributors']['links']['related']['href']
  authorRequest = requests.get( url = authorApiUrl )
  authorRequestContent = json.loads( authorRequest.content )
  
  for preprintAuthor in authorRequestContent['data']:
    print( preprintAuthor['embeds']['users']['data']['links']['html'] )
    print( preprintAuthor['embeds']['users']['data']['attributes']['full_name'] )
    
  print( '\n' )
    
print( time.time - ti )

2019-07-30T16:02:54.077336
SUB-CHRONIC MPTP-INDUCED PARKINSONISM: SELECTIVE SENSITIVITY OF THE  BALB/c MICE AND THE PROPHYLACTIC ROLES OF NIGELLA SATIVA OIL
https://osf.io/preprints/africarxiv/v7fu4/
https://doi.org/10.31730/osf.io/v7fu4
Life Sciences
Neuroscience and Neurobiology
Behavioral Neurobiology


Life Sciences
Neuroscience and Neurobiology
Cognitive Neuroscience


Medicine and Health Sciences
Alternative and Complementary Medicine


Life Sciences
Biology


https://osf.io/zq7sy/
Royhaan Folarin
https://osf.io/s6jrk/
Akinola Olonade
https://osf.io/w4zbc/
Praise Ogunkunle
https://osf.io/fgwhd/
Philemon Shallie
https://osf.io/pz6wn/
Thomas Adenowo
https://osf.io/y6gxa/
Muinat Adeyanju
https://osf.io/58gw2/
Paul Folarin
https://osf.io/9vrjf/
Tarilayefa Amangele


2019-07-15T08:40:18.925434
Book review: Urban environments in Africa - A critical analysis of environmental politics
https://osf.io/preprints/africarxiv/hu9xv/
https://doi.org/10.31730/osf.io/hu9xv
Social and Behavioral S

TypeError: ignored